# Pregunta 3: Desafío Kaggle

## Obtención de los datos

En esta sección se instala la utilidad de lína de comandos de Kaggle, se configuran las credenciales de acceso y se obtienen los datos de entrenamiento y pruebas del desafío.

In [0]:
!pip install -q kaggle

In [308]:
!mkdir ~/.kaggle
!wget -O ~/.kaggle/kaggle.json http://vps.csaldias.cl/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists
--2018-08-31 21:07:22--  http://vps.csaldias.cl/kaggle.json
Resolving vps.csaldias.cl (vps.csaldias.cl)... 170.239.86.99
Connecting to vps.csaldias.cl (vps.csaldias.cl)|170.239.86.99|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64 [application/json]
Saving to: ‘/root/.kaggle/kaggle.json’

/root/.kaggle/kaggl 100%[===================>]      64  --.-KB/s    in 0s      

2018-08-31 21:07:22 (6.13 MB/s) - ‘/root/.kaggle/kaggle.json’ saved [64/64]



In [3]:
!kaggle competitions download -c tareaANN

  0%|                                               | 0.00/11.9k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 11.9k/11.9k [00:00<00:00, 6.85MB/s]
  0%|                                               | 0.00/2.25k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 2.25k/2.25k [00:00<00:00, 1.93MB/s]
 96%|██████████████████████████████████████▌ | 339M/352M [00:04<00:00, 43.7MB/s]
100%|████████████████████████████████████████| 352M/352M [00:04<00:00, 81.2MB/s]
100%|██████████████████████████████████████▉| 1.37G/1.37G [00:10<00:00, 111MB/s]
100%|███████████████████████████████████████| 1.37G/1.37G [00:10<00:00, 139MB/s]
 54%|████████████████████▋                 | 9.00M/16.5M [00:00<00:00, 24.0MB/s]
100%|██████████████████████████████████████| 16.5M/16.5M [00:00<00:00, 35.9MB/s]
 91%|██████████████████████████████████▌   | 60.0M/66.0M [00:00<00:00, 71.8MB/s]
100%|███████████████████████████████████████| 66.0M/66.0M [00:00<00:00, 112MB/s]


In [4]:
!ls -lh

total 1.8G
-rw-r--r-- 1 root root  17M Aug 31 18:34 frames_test.tar.gz
-rw-r--r-- 1 root root  66M Aug 31 18:34 frames_train.tar.gz
-rw-r--r-- 1 root root 352M Aug 31 18:33 images_test.npy
-rw-r--r-- 1 root root 1.4G Aug 31 18:34 images_train.npy
-rw-r--r-- 1 root root  12K Aug 31 18:33 labels_train.csv
drwxr-xr-x 2 root root 4.0K Aug 30 21:39 sample_data
-rw-r--r-- 1 root root 2.3K Aug 31 18:33 sample_submision.csv


## Carga y limpieza de los datos

En esta sección se importan los datos de entrenamiento, y se realizan los preprocesamientos necesarios.

In [0]:
import numpy as np
import pandas as pd

x_train = np.load('images_train.npy')
df_train = pd.read_csv('labels_train.csv')

En este caso, decidimos escalar cada imagen a un 25% de su tamaño original, para agilizar el trabajo con la red y reducir su complejidad.

Nosotros hemos decidido abordar este desafío como un problema de clasificación, esto es, la red deberá decidir la categoría a la que pertenece cada imagen que sea probada o evaluada. En este caso, una categoría corresponde a una cantidad determinada de personas en una imagen.

Dado que en este dataset existe un máximo de 50 personas en una imagen (al menos en entrenamiento), la red tendrá que discernir entre 51 categorías (de 0 a 50 personas en la imagen, incluídas ambas cantidades).

In [0]:
from keras.utils import to_categorical
from skimage.transform import rescale

x_train_rescaled = []
for image in x_train:
  x_train_rescaled.append(rescale(image, 1.0 / 4.0))
x_train_rescaled = np.asarray(x_train_rescaled)

y_train = to_categorical(df_train['count'], 51)

In [366]:
x_train_rescaled.shape

(1600, 120, 160, 3)

## Definición de la CNN

En esta sección se define la arquitectura de la red a utilizar para resolver el desafío.

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.constraints import maxnorm

In [0]:
model = Sequential()

model.add(Conv2D(128, (5, 5), activation='relu', padding='same', input_shape=x_train_rescaled.shape[1:]))
model.add(Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(51, activation='softmax'))

Esta red está basada en la red CNN utilizada con el dataset CIFAR-10 en la Tarea 1 de este curso, con una arquitectura $C \times C \times P \times C \times C \times P \times F \times F$, dado que (como vimos anteriormente) estas redes tienen un buen desempeño en la clasificación de imágenes. Para este caso en particualr, decidimos adoptar una serie de modificaciones menores a la arquitercura de la red:

*  Se añadió una capa de BatchNormalization entre la última capa de pooling y las capas fully-connected, para mejorar el desempeño de la red y reducir las variaciones en el aprendizaje de la red.
*   Se añadió una segunda capa densa fully-connected al final de la red, para aumentar el número de parámetros entrenables y mejorar el desempeño sobre el dataset.
*   Se modificaron las cantidades de filtros en la primera tanda de capas convolucionales, así como el tamaño del kernel en la primera capa convolucional.
*   Se decidió utilizar *strides* en la capa convolucional antes del pooling.



In [370]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 120, 160, 128)     9728      
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 60, 80, 64)        73792     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 30, 40, 64)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 30, 40, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 30, 40, 64)        36928     
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 15, 20, 64)        36928     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 7, 10, 64)         0         
__________

## Entrenamiento de la CNN

En esta sección se realiza el entrenamiento de la red creada en la sección anterior.

In [0]:
from keras.optimizers import SGD, rmsprop, Adam
import math

In [0]:
learning_rate = 0.5
decay_rate = 1e-6

opt = rmsprop(lr=0.001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [373]:
epochs=20
batch_size=32
history=model.fit(x_train_rescaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, shuffle=True, verbose=2)

Train on 1440 samples, validate on 160 samples
Epoch 1/20
 - 9s - loss: 3.6053 - acc: 0.0556 - val_loss: 3.2953 - val_acc: 0.0375
Epoch 2/20
 - 8s - loss: 3.4236 - acc: 0.0514 - val_loss: 3.2175 - val_acc: 0.0688
Epoch 3/20
 - 8s - loss: 3.2631 - acc: 0.0667 - val_loss: 5.0410 - val_acc: 0.0250
Epoch 4/20
 - 8s - loss: 3.1079 - acc: 0.0944 - val_loss: 10.2717 - val_acc: 0.0125
Epoch 5/20
 - 8s - loss: 3.0187 - acc: 0.1056 - val_loss: 6.5602 - val_acc: 0.0125
Epoch 6/20
 - 8s - loss: 2.8412 - acc: 0.1396 - val_loss: 4.4919 - val_acc: 0.0375
Epoch 7/20
 - 8s - loss: 2.7251 - acc: 0.1556 - val_loss: 8.9038 - val_acc: 0.0063
Epoch 8/20
 - 8s - loss: 2.5804 - acc: 0.1882 - val_loss: 6.2819 - val_acc: 0.0000e+00
Epoch 9/20
 - 8s - loss: 2.3416 - acc: 0.2576 - val_loss: 7.6474 - val_acc: 0.0000e+00
Epoch 10/20
 - 8s - loss: 2.1037 - acc: 0.3250 - val_loss: 4.7859 - val_acc: 0.0250
Epoch 11/20
 - 8s - loss: 1.8862 - acc: 0.3903 - val_loss: 10.3505 - val_acc: 0.0437
Epoch 12/20
 - 8s - loss: 1.

In [378]:
epochs=5
batch_size=32
history=model.fit(x_train_rescaled, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, shuffle=True, verbose=2)

Train on 1440 samples, validate on 160 samples
Epoch 1/5
 - 8s - loss: 0.1825 - acc: 0.9472 - val_loss: 11.8548 - val_acc: 0.0938
Epoch 2/5
 - 8s - loss: 0.1621 - acc: 0.9507 - val_loss: 6.1652 - val_acc: 0.1125
Epoch 3/5
 - 8s - loss: 0.1542 - acc: 0.9493 - val_loss: 6.3710 - val_acc: 0.0813
Epoch 4/5
 - 8s - loss: 0.1464 - acc: 0.9479 - val_loss: 12.6346 - val_acc: 0.0312
Epoch 5/5
 - 8s - loss: 0.1153 - acc: 0.9667 - val_loss: 7.5933 - val_acc: 0.0750


## Pruebas sobre la Red

Como una medida del desempeño aproximado de la red, medimos el error RMSE sobre el dataset de entrenamiento completo.

In [0]:
Y_trained = model.predict(x_train_rescaled,batch_size=1)

In [0]:
y_trained_argmax = []
y_argmax = []
for elem in Y_trained:
  y_trained_argmax.append(np.argmax(elem))
for elem in y_train:
  y_argmax.append(np.argmax(elem))

In [381]:
from sklearn.metrics import mean_squared_error
math.sqrt(mean_squared_error(y_argmax, y_trained_argmax))

1.485555451674558

Ocasionalmente, guardamos el modelo entrenado. Esto nos permitió probar con distintas arquitecturas y volver a otras ya probadas en caso de que algo saliera mal.

In [0]:
model.save('model-1.485.h5')

## Preparación de Submission a Kaggle

El siguiente código importa las imágenes de prueba, realiza el mismo escalado que realizamos para el conjunto de entrenamiento (25% del tamaño original), obtiene las predicciones de la red en base a las imágenes de prueba, y crea el archivo CSV que posteriormente es descargado para ser subido a Kaggle.

In [0]:
x_test = np.load('images_test.npy')
x_test_rescaled = []
for image in x_test:
  x_test_rescaled.append(rescale(image, 1.0 / 4.0))
x_test_rescaled = np.asarray(x_test_rescaled)

In [0]:
Y_trained = model.predict(x_test_rescaled, batch_size=1)

In [0]:
y_trained_argmax = []
for elem in Y_trained:
  y_trained_argmax.append(np.argmax(elem))

In [0]:
test_ids = list(range(1, 401))
d = {'id': test_ids, 'count': y_trained_argmax}
entrega = pd.DataFrame(data=d,columns=['id','count'])
entrega.to_csv('submission-1.485.csv', index=False)